In [13]:
import spacy
import json


from spacy.tokens import Doc
from spacy.pipeline import EntityRuler
from spacy import displacy
from pprint import pprint

from nav_pii_anon.regex_parser import RegexEngines

In [11]:
# Ikke ekte personopplysninger

text = """Hei, mitt navn er Peter Newfield (personnummer: 211132 32983)
og jeg kommer fra Bergen. Jeg har prøv å komme i kontakt med NAV en stund nå,
men har ikke fått svar på mine henvendeler angående saken min (saksnummer: 174920457).
Hvis jeg ikke får dagpengene mine snart, kommer jeg til å bli kastet ut av Sporveisgata 10, 0354 Oslo.
Dere kan kontakte meg på 99233190 eller på peternewfield@gmail.com."""

In [3]:
def get_patterns():
    patterns  = []
    regex_engines = [engine.value for engine in RegexEngines]
    
    for engine in regex_engines:
        patterns.append({"label": engine.label, "pattern": [{"TEXT": {"REGEX":  engine.regex_pattern}}]})
    
    
    return patterns

In [4]:
def save_patterns(file_name: str, patterns:list):
    with open(file_name, 'w') as file:
        json.dump(patterns, file)

In [5]:
def add_regex_pipe(nlp, patterns: list, name: str):
    regex_pipe = EntityRuler(nlp)
    regex_pipe.add_patterns(patterns)
    nlp.add_pipe(regex_pipe, name )

In [6]:
def get_ents(doc: Doc):
    ents = []
    for ent in doc.ents:
        ents.append({
            'text': ent.text,
            'label': ent.label_,
            'start': ent.start_char,
            'end': ent.end_char
        })
    
    return ents

In [7]:
def parse_text(text: str, nlp = None, pipe_name: str = None, vizualize: bool = False):
    
    if not nlp:
        nlp = spacy.load("nb_core_news_lg")
    
    if not pipe_name:
        pipe_name = "regex"
    
        
    patterns = get_patterns()
    add_regex_pipe(nlp, patterns, pipe_name)
    
    doc = nlp(text)
    
    displacy.render(doc, style="ent")
    
    return get_ents(doc)

In [15]:
nlp = spacy.load("nb_core_news_lg")

In [16]:
type(nlp)

spacy.lang.nb.Norwegian

In [12]:
parse_text(text)

[{'text': 'Peter Newfield', 'label': 'PER', 'start': 18, 'end': 32},
 {'text': 'Bergen', 'label': 'LOC', 'start': 80, 'end': 86},
 {'text': 'NAV', 'label': 'ORG', 'start': 123, 'end': 126},
 {'text': 'Sporveisgata', 'label': 'LOC', 'start': 302, 'end': 314},
 {'text': 'Oslo', 'label': 'LOC', 'start': 324, 'end': 328},
 {'text': 'peternewfield@gmail.com',
  'label': 'MISC',
  'start': 373,
  'end': 396}]